In [1]:
import paddle
import paddle.nn.functional as F
import numpy as np
import pandas as pd
import os
from paddle import nn
from PIL import Image
from paddle.distributed import fleet, get_rank

IMAGE_SIZE = 224
BATCH_SIZE = 4
EPOCH_NUM = 5

print(paddle.__version__)

2.4.1


In [2]:
# 定义数据集
class MyDataset(paddle.io.Dataset):
    def __init__(self, img_dir='data/PALM-Training400/', csv_dir='data/Classification.csv') -> None:
        super(MyDataset, self).__init__()
        if csv_dir is None:
            self.csvfile = None
            self.filedir = os.listdir(img_dir)
        else:
            self.csvfile = pd.read_csv(csv_dir)
        self.imgpath = img_dir
        pass
    def __len__(self):
        if self.csvfile is None:
            return len(self.filedir)
        else:
            return len(self.csvfile)
        pass
    def __getitem__(self, idx):
        if self.csvfile is None:
            img = np.reshape((np.array(Image.open(self.imgpath+os.sep+self.filedir[idx]).resize((IMAGE_SIZE,IMAGE_SIZE))).astype('float32')),(3,IMAGE_SIZE,IMAGE_SIZE))/256.
            lab = self.filedir[idx]
        else:
            img = np.reshape((np.array(Image.open(self.imgpath+os.sep+self.csvfile['imgName'][idx]).resize((IMAGE_SIZE,IMAGE_SIZE))).astype('float32')),(3,IMAGE_SIZE,IMAGE_SIZE))/256.
            lab = np.array(self.csvfile['Label'][idx]).astype('float32')
        return img,lab
    pass
mydataset = MyDataset()

In [3]:
import paddle
import numpy as np
from paddle.nn import Conv2D, MaxPool2D, Linear, Dropout
## 组网
import paddle.nn.functional as F

# 定义 AlexNet 网络结构
class AlexNet(paddle.nn.Layer):
    def __init__(self, num_classes=1):
        super(AlexNet, self).__init__()
        # AlexNet与LeNet一样也会同时使用卷积和池化层提取图像特征
        # 与LeNet不同的是激活函数换成了‘relu’
        self.conv1 = Conv2D(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=5)
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        self.conv2 = Conv2D(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        self.conv3 = Conv2D(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv4 = Conv2D(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = Conv2D(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.max_pool5 = MaxPool2D(kernel_size=2, stride=2)

        self.fc1 = Linear(in_features=12544, out_features=4096)
        self.drop_ratio1 = 0.5
        self.drop1 = Dropout(self.drop_ratio1)
        self.fc2 = Linear(in_features=4096, out_features=4096)
        self.drop_ratio2 = 0.5
        self.drop2 = Dropout(self.drop_ratio2)
        self.fc3 = Linear(in_features=4096, out_features=num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.conv5(x)
        x = F.relu(x)
        x = self.max_pool5(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc1(x)
        x = F.relu(x)
        # 在全连接之后使用dropout抑制过拟合
        x = self.drop1(x)
        x = self.fc2(x)
        x = F.relu(x)
        # 在全连接之后使用dropout抑制过拟合
        x = self.drop2(x)
        x = self.fc3(x)
        return x
model = AlexNet()

W0203 20:42:45.433202 358030 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 6.1, Driver API Version: 12.0, Runtime API Version: 11.7
W0203 20:42:45.452216 358030 gpu_resources.cc:91] device: 0, cuDNN Version: 8.7.


In [4]:
def train_pm(model, optimizer):
    # 开启0号GPU训练
    paddle.device.set_device('gpu:0')

    print('start training ... ')
    model.train()
    # 定义数据读取器，训练数据读取器和验证数据读取器
    train_loader = paddle.io.DataLoader(mydataset,batch_size=BATCH_SIZE,shuffle=True,drop_last=True)
    valid_loader = paddle.io.DataLoader(mydataset,batch_size=BATCH_SIZE,shuffle=True,drop_last=True)
    for epoch in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            x_data, y_data = data
            img = paddle.to_tensor(x_data)
            label = paddle.reshape(paddle.to_tensor(y_data),(-1,1))
            # 运行模型前向计算，得到预测值
            logits = model(img)
            loss = F.binary_cross_entropy_with_logits(logits, label)
            avg_loss = paddle.mean(loss)

            if batch_id % 20 == 19:
                print("epoch: {}, batch_id: {}, loss is: {:.4f}".format(epoch+1, batch_id+1, float(avg_loss.numpy())))
            # 反向传播，更新权重，清除梯度
            avg_loss.backward()
            optimizer.step()
            optimizer.clear_grad()

        model.eval()
        accuracies = []
        losses = []
        for batch_id, data in enumerate(valid_loader()):
            x_data, y_data = data
            img = paddle.to_tensor(x_data)
            label = paddle.reshape(paddle.to_tensor(y_data),(-1,1))
            # 运行模型前向计算，得到预测值
            logits = model(img)
            # 二分类，sigmoid计算后的结果以0.5为阈值分两个类别
            # 计算sigmoid后的预测概率，进行loss计算
            pred = F.sigmoid(logits)
            loss = F.binary_cross_entropy_with_logits(logits, label)
            # 计算预测概率小于0.5的类别
            pred2 = pred * (-1.0) + 1.0
            # 得到两个类别的预测概率，并沿第一个维度级联
            pred = paddle.concat([pred2, pred], axis=1)
            acc = paddle.metric.accuracy(pred, paddle.cast(label, dtype='int64'))

            accuracies.append(acc.numpy())
            losses.append(loss.numpy())
        print("[validation] accuracy/loss: {:.4f}/{:.4f}".format(np.mean(accuracies), np.mean(losses)))
        model.train()

        paddle.save(model.state_dict(), 'model/palmp.pdparams')
        paddle.save(optimizer.state_dict(), 'model/palmp.pdopt')

In [7]:
opt = paddle.optimizer.Momentum(learning_rate=0.001, momentum=0.9, parameters=model.parameters())

# 启动训练过程
train_pm(model, opt)

start training ... 
epoch: 1, batch_id: 20, loss is: 1.0326
epoch: 1, batch_id: 40, loss is: 0.5023
epoch: 1, batch_id: 60, loss is: 0.6070
epoch: 1, batch_id: 80, loss is: 0.4975
epoch: 1, batch_id: 100, loss is: 0.1995
epoch: 1, batch_id: 120, loss is: 0.1507
epoch: 1, batch_id: 140, loss is: 0.0392
epoch: 1, batch_id: 160, loss is: 0.4684
epoch: 1, batch_id: 180, loss is: 0.2646
epoch: 1, batch_id: 200, loss is: 0.2130
[validation] accuracy/loss: 0.9000/0.2542
epoch: 2, batch_id: 20, loss is: 0.0728
epoch: 2, batch_id: 40, loss is: 0.0846
epoch: 2, batch_id: 60, loss is: 0.1180
epoch: 2, batch_id: 80, loss is: 0.0045
epoch: 2, batch_id: 100, loss is: 0.0247
epoch: 2, batch_id: 120, loss is: 0.3566
epoch: 2, batch_id: 140, loss is: 0.3164
epoch: 2, batch_id: 160, loss is: 0.0617
epoch: 2, batch_id: 180, loss is: 1.0732
epoch: 2, batch_id: 200, loss is: 0.1328
[validation] accuracy/loss: 0.9400/0.1629
epoch: 3, batch_id: 20, loss is: 0.6663
epoch: 3, batch_id: 40, loss is: 0.0103
epoc

In [5]:
def predict(model, dataloader):
    model.eval()
    pred_list = np.array([])
    file_list = np.array([])
    for idx, data in enumerate(dataloader):
        print('\r{}/{}'.format(1+idx,len(dataloader)),end='')
        x_data, filename = data
        img = paddle.to_tensor(x_data)
        # 运行模型前向计算，得到预测值
        logits = model(img)
        # 二分类，sigmoid计算后的结果以0.5为阈值分两个类别
        # 计算sigmoid后的预测概率，进行loss计算
        pred = F.sigmoid(logits)
        pred_list = np.append(pred_list, pred.numpy().ravel())
        file_list = np.append(file_list, filename)
    return pred_list, file_list

In [6]:
# 读取模型
model = AlexNet()
model.set_state_dict(paddle.load('model/palmp.pdparams'))
testdataset = MyDataset('data/PALM-Testing400',None)
testdataloader = paddle.io.DataLoader(testdataset,shuffle=False,drop_last=False,batch_size=BATCH_SIZE)

pred_list,file_list = predict(model, testdataloader)
pd.DataFrame(np.c_[file_list,pred_list],columns=['FileName','PM Risk']).to_csv('Classification_Results.csv',index=False)

100/100